In [46]:
import pandas as pd

data = pd.read_csv(f'data/Power-Networks-LCL-June2015(withAcornGps)v2_1.csv', parse_dates=[2], na_values=['Null'])
for i in range(2,25):
    data = data.append(pd.read_csv(f'data/Power-Networks-LCL-June2015(withAcornGps)v2_{i}.csv', parse_dates=[2], na_values=['Null']))
    
data.columns = ['houseid', 'std_flag', 'datetime', 'consumption', 'acorn', 'acorn_grouped']
data.consumption = data.consumption.astype(float)
data.consumption *= 2 # to get kW
data.drop(columns=['std_flag', 'acorn', 'acorn_grouped'], inplace=True)
data.drop_duplicates(['houseid', 'datetime'], inplace=True)

In [63]:
import datetime
non_null_counts = data[(data.consumption > 0) & (data.datetime >= datetime.datetime(2013,1,1)) & (data.datetime < datetime.datetime(2014,1,1))].groupby('houseid').agg('count').sort_values('consumption')
relevant_houses = non_null_counts[non_null_counts.consumption >= 17500]

In [83]:
filtered_data = data[data['houseid'].isin(relevant_houses.index[-500:])]
filtered_data = filtered_data[(datetime.datetime(2013,1,1) <= filtered_data.datetime) & (filtered_data.datetime < datetime.datetime(2014,1,1))]
filtered_data.reset_index(inplace=True)
filtered_data.drop(columns=['index'], inplace=True)

In [112]:
import matplotlib.pyplot as plt
%matplotlib inline

consumptions = pd.pivot_table(filtered_data, index=['datetime', 'houseid']).unstack()

In [121]:
import numpy as np

houseids = list(consumptions.columns)
selected = np.random.choice(consumptions.columns, 50)
baseline = list(set(houseids) - set(selected))

consumptions[selected].to_csv('selected_5.csv')
consumptions[baseline].sum(axis=1).to_csv('baseline_5.csv')